<a href="https://colab.research.google.com/github/sahandv/science_science/blob/master/clustering_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf 'science_science'
username = "sahandv"#@param {type:"string"}
# password = ""#@param {type:"string"} 
!git clone https://github.com/$username/science_science.git
!ls

Cloning into 'science_science'...
remote: Enumerating objects: 317, done.
remote: Counting objects: 100% (317/317), done.
remote: Compressing objects: 100% (279/279), done.
remote: Total 1319 (delta 178), reused 162 (delta 37), pack-reused 1002
Receiving objects: 100% (1319/1319), 102.74 MiB | 41.22 MiB/s, done.
Resolving deltas: 100% (773/773), done.
sample_data  science_science


In [2]:
from google.colab import drive
drive.mount('/content/drive/')
!pip install -r 'science_science/requirements.txt'
# !pip install tensorflow
# !pip install keras

Mounted at /content/drive/
     |████████████████████████████████| 1.6MB 16.8MB/s 
     |████████████████████████████████| 40kB 7.2MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=703d43501249594b447aa6e7f8d588c0ab5c17937f441d0a3369baefdb284b15
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for netgraph: filename=netgraph-3.1.8-cp36-none-any.whl size=45928 sha256=daa4b6a34f841e201bf4c563e7022f46fad13770c83ec5cd3d28f3353d450931
  Stored in directory: /root/.cache/pip/wheels/a1/c2/49/30a980fcf7c864676d6184e46ea8860fff2b7fa6f324cad69a
Successfully built pyLDAvis netgraph


In [3]:
# datapath = '/mnt/6016589416586D52/Users/z5204044/GoogleDrive/GoogleDrive/Data/' # Local C1314
# datapath = '/mnt/16A4A9BCA4A99EAD/GoogleDrive/Data/' # Local Ryzen
datapath = 'drive/My Drive/Data/' # Remote

In [4]:
# Remote
from science_science.sciosci.assets import text_assets as ta
from science_science.DEC.DEC_keras import DEC_simple_run

# Local
# from sciosci.assets import text_assets as ta
# from DEC.DEC_keras import DEC_simple_run

## Sequential test

In [ ]:
import sys
import time
import gc
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from random import randint

from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
from sklearn import metrics
from sklearn.metrics.cluster import silhouette_score,homogeneity_score,adjusted_rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score,adjusted_mutual_info_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer , TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing


# =============================================================================
# Load data and init
# =============================================================================
data_address =  datapath+"Corpus/KPRIS/embeddings/deflemm/bert scibert corpus.csv"
label_address =  datapath+"Corpus/KPRIS/labels"

vectors = pd.read_csv(data_address)
labels = pd.read_csv(label_address,names=['label'])
labels_f = pd.factorize(labels.label)
X = vectors.values
Y = labels_f[0]
n_clusters = 5

labels_task_1 = labels[(labels['label']=='car') | (labels['label']=='memory')]
vectors_task_1 = vectors.iloc[labels_task_1.index]
labels_task_1_f = pd.factorize(labels_task_1.label)
X_task_1 = vectors_task_1.values
Y_task_1 = labels_task_1_f[0]
n_clusters_task_1 = 2

results = pd.DataFrame([],columns=['Method','parameter','Silhouette','Homogeneity','NMI','AMI','ARI'])
# =============================================================================
# Evaluation method
# =============================================================================
def evaluate(X,Y,predicted_labels):
    
    df = pd.DataFrame(predicted_labels,columns=['label'])
    if len(df.groupby('label').groups)<2:
        return [0,0,0,0,0]
    
    return [silhouette_score(X, predicted_labels, metric='euclidean'),
                    homogeneity_score(Y, predicted_labels),
                    normalized_mutual_info_score(Y, predicted_labels),
                    adjusted_mutual_info_score(Y, predicted_labels),
                    adjusted_rand_score(Y, predicted_labels)]

In [ ]:

# =============================================================================
# K-means
# =============================================================================
print('\n- k-means random -----------------------')
for fold in tqdm(range(20)):
    seed = randint(0,10**5)
    model = KMeans(n_clusters=n_clusters,n_init=20, init='random', random_state=seed).fit(X)
    predicted_labels = model.labels_
    tmp_results = ['k-means random','seed '+str(seed)]+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# K-means with init='k-means++'
# =============================================================================
print('\n- k-means++ -----------------------')
for fold in tqdm(range(20)):
    seed = randint(0,10**5)
    model = KMeans(n_clusters=n_clusters,n_init=20,init='k-means++', random_state=seed).fit(X)
    predicted_labels = model.labels_
    tmp_results = ['k-means++','seed '+str(seed)]+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# Agglomerative
# =============================================================================
print('\n- Agglomerative -----------------------')
for fold in tqdm(range(4)):
    model = AgglomerativeClustering(n_clusters=n_clusters,linkage='ward').fit(X)
    predicted_labels = model.labels_
    tmp_results = ['Agglomerative','ward']+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# DBSCAN
# =============================================================================
eps=0.000001
print('\n- DBSCAN -----------------------')
for fold in tqdm(range(19)):
    eps = eps+0.05
    model = DBSCAN(eps=eps, min_samples=10,n_jobs=15).fit(X)
    predicted_labels = model.labels_
    tmp_results = ['DBSCAN','eps '+str(eps)]+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# Deep no min_max_scaling
# =============================================================================
archs = [[500, 500, 2000, 10],[500, 1000, 2000, 10],[500, 1000, 1000, 10],
         [500, 500, 2000, 100],[500, 1000, 2000, 100],[500, 1000, 1000, 100],
         [100, 300, 600, 10],[300, 500, 2000, 10],[700, 1000, 2000, 10],
         [200, 500, 10],[500, 1000, 10],[1000, 2000, 10],
         [200, 500, 100],[500, 1000, 100],[1000, 2000, 100],
         [1000, 500, 10],[500, 200, 10],[200, 100, 10],
         [1000, 1000, 2000, 10],[1000, 1500, 2000, 10],[1000, 1500, 1000, 10],
         [1000, 1000, 2000,500, 10],[1000, 1500, 2000,500, 10],[1000, 1500, 1000, 500, 10],
         [500, 500, 2000, 500, 10],[500, 1000, 2000, 500, 10],[500, 1000, 1000, 500, 10]]
print('\n- DEC -----------------------')
for fold in tqdm(archs):
    seed = randint(0,10**4)
    np.random.seed(seed)
    predicted_labels = DEC_simple_run(X,minmax_scale_custom_data=False,n_clusters=5,architecture=fold,pretrain_epochs=400)
    tmp_results = ['DEC',str(seed)+' '+str(fold)]+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# Deep with min_max_scaling
# =============================================================================
archs = [[500, 500, 2000, 10],[500, 1000, 2000, 10],[500, 1000, 1000, 10],
         [500, 500, 2000, 100],[500, 1000, 2000, 100],[500, 1000, 1000, 100],
         [100, 300, 600, 10],[300, 500, 2000, 10],[700, 1000, 2000, 10],
         [200, 500, 10],[500, 1000, 10],[1000, 2000, 10],
         [200, 500, 100],[500, 1000, 100],[1000, 2000, 100],
         [1000, 500, 10],[500, 200, 10],[200, 100, 10],
         [1000, 1000, 2000, 10],[1000, 1500, 2000, 10],[1000, 1500, 1000, 10],
         [1000, 1000, 2000,500, 10],[1000, 1500, 2000,500, 10],[1000, 1500, 1000, 500, 10],
         [500, 500, 2000, 500, 10],[500, 1000, 2000, 500, 10],[500, 1000, 1000, 500, 10]]
print('\n- DEC -----------------------')
for fold in tqdm(archs):
    seed = randint(0,10**4)
    np.random.seed(seed)
    predicted_labels = DEC_simple_run(X,minmax_scale_custom_data=False,n_clusters=5,architecture=fold,pretrain_epochs=400)
    tmp_results = ['DEC',str(seed)+' '+str(fold)]+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)


In [ ]:
# =============================================================================
# Save to disk
# =============================================================================
results_df = pd.DataFrame(results)
results_df.to_csv(data_address+' clustering results _ new',index=False)

Mini test

In [ ]:
# =============================================================================
# Deep no min_max_scaling
# =============================================================================
archs = [[200, 1000, 2000,100],[200, 1000, 2000,200],[200, 1000, 2000, 500],
            [200, 500, 1000, 500],[200, 500, 1000, 200],[200, 500, 1000, 100],
            [200, 1000, 2000, 10],[200, 1000, 2000, 10],[200, 1000, 2000, 10],
            [1536,3072,1536,100],[1536,3072,1536,100],[1536,3072,1536,100],
            [1536,3072,1536,10],[1536,3072,1536,10],[1536,3072,1536,10],
            [1536,3072,1536,100,10],[1536,3072,1536,100,10],[1536,3072,1536,100,10],
            [1536,3072,1536,100],[1536,3072,1536,100],[1536,3072,1536,100],
            [1536,3072,1536,10],[1536,3072,1536,10],[1536,3072,1536,10],
            [1536,3072,1536,100,10],[1536,3072,1536,100,10],[1536,3072,1536,100,10]]

print('\n- DEC -----------------------')
for fold in tqdm(archs):
    seed = randint(0,10**4)
    np.random.seed(seed)
    try:
        predicted_labels = DEC_simple_run(X,minmax_scale_custom_data=False,n_clusters=5,architecture=fold,pretrain_epochs=400)
        tmp_results = ['DEC',str(seed)+' '+str(fold)]+evaluate(X,Y,predicted_labels)
        tmp_results = pd.Series(tmp_results, index = results.columns)
        results = results.append(tmp_results, ignore_index=True)
    except:
        print('Some error happened, skipping ',fold)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# Save to disk
# =============================================================================
results_df = pd.DataFrame(results)
results_df.to_csv(data_address+' clustering results _ [200, 1000, 2000,100]',index=False)

## Functional test

In [5]:
import sys
import time
import gc
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from random import randint

from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
from sklearn import metrics
from sklearn.metrics.cluster import silhouette_score,homogeneity_score,adjusted_rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score,adjusted_mutual_info_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer , TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing

from science_science.sciosci.assets import text_assets as ta
from science_science.DEC.DEC_keras import DEC_simple_run

In [ ]:
data_dir =  datapath+"Corpus/cora-classify/cora/"
label_address =  datapath+"Corpus/cora-classify/cora/corpus classes1 with citations"

vec_file_names = ['embeddings/node2vec deepwalk-100-6-100 p1q1','embeddings/node2ve deepwalk 80-10-128']#,'Doc2Vec patent corpus',
                #   ,'embeddings/node2vec-80-10-128 p1q0.5','embeddings/node2vec-80-10-128 p4q1'],
                  # 'FastText SIF patent_wos corpus','FastText SIF wos corpus']
labels = pd.read_csv(label_address,names=['label'])

clusters = labels.groupby('label').groups

for file_name in vec_file_names:
    gc.collect()
    output_file_name = data_dir+file_name+' clustering results'
    run_all_tests(data_dir+file_name,output_file_name,labels,len(list(clusters.keys())))

Will write the results to drive/My Drive/Data/Corpus/cora-classify/cora/embeddings/Doc2Vec cora corpus dm1 with citations clustering results


  0%|          | 0/39 [00:00<?, ?it/s]


- DEC -----------------------
Model: "AE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 100)]             0         
_________________________________________________________________
encoder_0 (Dense)            (None, 500)               50500     
_________________________________________________________________
encoder_1 (Dense)            (None, 500)               250500    
_________________________________________________________________
encoder_2 (Dense)            (None, 2000)              1002000   
_________________________________________________________________
encoder_3 (Dense)            (None, 500)               1000500   
_________________________________________________________________
decoder_3 (Dense)            (None, 2000)              1002000   
_________________________________________________________________
decoder_2 (Dense)            (Non

In [6]:

# =============================================================================
# Evaluation method
# =============================================================================
def evaluate(X,Y,predicted_labels):
    
    df = pd.DataFrame(predicted_labels,columns=['label'])
    if len(df.groupby('label').groups)<2:
        return [0,0,0,0,0]
    
    try:
        sil = silhouette_score(X, predicted_labels, metric='euclidean')
    except:
        sil = 0
        
    return [sil,
            homogeneity_score(Y, predicted_labels),
            normalized_mutual_info_score(Y, predicted_labels),
            adjusted_mutual_info_score(Y, predicted_labels),
            adjusted_rand_score(Y, predicted_labels)]
# =============================================================================
# Cluster and evaluate
# =============================================================================
def run_all_tests(data_address,output_file_name,labels,k):
    print('k=',k)
    print('Will write the results to',output_file_name)
    
    column_names = ['Method','parameter','Silhouette','Homogeneity','NMI','AMI','ARI']
    vectors = pd.read_csv(data_address)
    # data_dir+file_name+' dm_concat'
    labels_f = pd.factorize(labels.label)

    X = vectors.values
    Y = labels_f[0]
    n_clusters = k
    
    labels_task_1 = labels[(labels['label']=='car') | (labels['label']=='memory')]
    vectors_task_1 = vectors.iloc[labels_task_1.index]
    labels_task_1_f = pd.factorize(labels_task_1.label)
    X_task_1 = vectors_task_1.values
    Y_task_1 = labels_task_1_f[0]
    n_clusters_task_1 = 2
    
    results = pd.DataFrame([],columns=column_names)
    results_template = results.copy()
    # =============================================================================
    # Deep no min_max_scaling
    # =============================================================================
    archs = [
            [500, 500, 2000, 500],[500, 500, 2000, 500],[500, 500, 2000, 500],
            # [500, 500, 2000, 500],[500, 500, 2000, 500],[500, 500, 2000, 500],
            [500, 1000, 2000, 100],[500, 1000, 2000, 100],[500, 1000, 2000, 100],
            [200, 1000, 2000,100, 10],[200, 1000, 2000,200, 10],[200, 1000, 2000, 500, 10],
            # [200, 500, 1000, 500, 10],[200, 500, 1000, 200, 10],[200, 500, 1000, 100, 10],
            # [200, 1000, 2000,100, 10],[200, 1000, 2000,200, 10],[200, 1000, 2000, 500, 10],
            [200, 500, 1000, 500, 10],[200, 500, 1000, 200, 10],[200, 500, 1000, 100, 10],
            # [200, 1000, 2000,100],[200, 1000, 2000,200],[200, 1000, 2000, 500],
            # [200, 500, 1000, 500],[200, 500, 1000, 200],[200, 500, 1000, 100],
            # [200, 1000, 2000, 10],[200, 1000, 2000, 10],[200, 1000, 2000, 10],
            [1536,3072,1536,100],[1536,3072,1536,100],[1536,3072,1536,100],
            [256,1024,512,10],[256,1024,512,10],[256,1024,512,10],
            [1024,1024,2048,256,10],[1024,1024,2048,256,10],
            [512,1024,2048,128,10],[512,1024,2048,128,10],
            [512,1024,2048,10],[512,1024,2048,10],
            [1024,1024,2048,10],[1024,1024,2048,10],
            # [1536,768,384,192,10],[1536,768,384,192,10],[1536,768,192,10],
            [1536,3072,1536,100,10],[1536,3072,1536,100,10],[1536,3072,1536,100,10],
            # [1536,3072,1536,100],[1536,3072,1536,100],[1536,3072,1536,100],
            # [1536,3072,1536,10],[1536,3072,1536,10],[1536,3072,1536,10],
            # [1536,3072,1536,100,10],[1536,3072,1536,100,10],[1536,3072,1536,100,10],
            # [200,200,100],[200,200,100],[200,200,100],
            [200,500,20],[200,500,200],[200,500,200],
            # [200,200,10],[200,200,10],[200,200,10],
            # [400,400,10],[400,400,10],[400,400,10],
            # [400,400,10],[400,400,10],[400,400,10],
            [400,1000,10],[400,1000,10],[400,1000,100,10],[400,1000,100,10],
            [400,500,10],[400,500,10],[400,500,10],
            # [200,200,10],[200,200,10],[200,200,10],
            # [200,200,10],[200,200,10],[200,200,10],
            [200,200,10],[200,200,10],[200,200,10],
            # [200,500,10],[200,500,10],[200,500,10],
            # [200,500,10],[200,500,10],[200,500,10],
            # [200,500,10],[200,500,10],[200,500,10],
            # [200,500,10],[200,500,10],[200,500,10],
            [200,500,10],[200,500,10],[200,500,10]]
    print('\n- DEC -----------------------')
    for fold in tqdm(archs):
        gc.collect()
        seed = randint(0,10**4)
        np.random.seed(seed)
        try:
            predicted_labels = DEC_simple_run(X,minmax_scale_custom_data=False,n_clusters=n_clusters,architecture=fold,pretrain_epochs=1000)
            tmp_results = ['DEC',str(seed)+' '+str(fold)]+evaluate(X,Y,predicted_labels)
            tmp_results_s = pd.Series(tmp_results, index = results.columns)
            tmp_results_df = results_template.copy()
            tmp_results_df = tmp_results_df.append(tmp_results_s, ignore_index=True)
            results = results.append(tmp_results_s, ignore_index=True)
        except:
            print('Some error happened, skipping ',fold)
        
        print('writing the fold results to file')
        # if file does not exist write header 
        try:
            if not os.path.isfile(output_file_name):
                tmp_results_df.to_csv(output_file_name, header=column_names,index=False)
            else: # else it exists so append without writing the header
                tmp_results_df.to_csv(output_file_name, mode='a', header=False,index=False)
        except:
            print('something went wrong and could not write the results to file!\n',
                 'You may abort and see what can be done.\n',
                 'Or wait to see the the final results in memory.')
        
    mean = results.mean(axis=0)
    maxx = results.max(axis=0)
    print(mean)
    print(maxx)
    
    # # =============================================================================
    # # Deep with min_max_scaling
    # # =============================================================================
    # archs = [[500, 500, 2000, 10],[500, 1000, 2000, 10],[500, 1000, 1000, 10],
    #             [500, 500, 2000, 100],[500, 1000, 2000, 100],[500, 1000, 1000, 100],
    #             [100, 300, 600, 10],[300, 500, 2000, 10],[700, 1000, 2000, 10],
    #             [200, 500, 10],[500, 1000, 10],[1000, 2000, 10],
    #             [200, 500, 100],[500, 1000, 100],[1000, 2000, 100],
    #             [1000, 500, 10],[500, 200, 10],[200, 100, 10],
    #             [1000, 1000, 2000, 10],[1000, 1500, 2000, 10],[1000, 1500, 1000, 10],
    #             [1000, 1000, 2000,500, 10],[1000, 1500, 2000,500, 10],[1000, 1500, 1000, 500, 10],
    #             [500, 500, 2000, 500, 10],[500, 1000, 2000, 500, 10],[500, 1000, 1000, 500, 10],
    #             [200,200,10],[200,200,10],[200,200,10],
    #             [200,200,10],[200,200,10],[200,200,10],
    #             [200,200,10],[200,200,10],[200,200,10],
    #             [200,500,10],[200,500,10],[200,500,10],
    #             [200,500,10],[200,500,10],[200,500,10]]
    # print('\n- DEC 2-----------------------')
    # for fold in tqdm(archs):
    #     predicted_labels = DEC_simple_run(X,minmax_scale_custom_data=True,n_clusters=5,architecture=fold,pretrain_epochs=300)
    #     tmp_results = ['DEC minmax scaler',str(fold)]+evaluate(X,Y,predicted_labels)
    #     tmp_results = pd.Series(tmp_results, index = results.columns)
    #     results = results.append(tmp_results, ignore_index=True)
    # mean = results.mean(axis=0)
    # maxx = results.max(axis=0)
    # print(mean)
    # print(maxx)
    

    # =============================================================================
    # K-means
    # =============================================================================
    print('\n- k-means random -----------------------')
    for fold in tqdm(range(20)):
        seed = randint(0,10**5)
        model = KMeans(n_clusters=n_clusters,n_init=20, init='random', random_state=seed).fit(X)
        predicted_labels = model.labels_
        try:
            tmp_results = ['k-means random','seed '+str(seed)]+evaluate(X,Y,predicted_labels)
            tmp_results_s = pd.Series(tmp_results, index = results.columns)
            tmp_results_df = results_template.copy()
            tmp_results_df = tmp_results_df.append(tmp_results_s, ignore_index=True)
            results = results.append(tmp_results_s, ignore_index=True)
        except:
            print('Some error happened, skipping ',fold)
        
        print('writing the fold results to file')
        # if file does not exist write header 
        try:
            if not os.path.isfile(output_file_name):
                tmp_results_df.to_csv(output_file_name, header=column_names,index=False)
            else: # else it exists so append without writing the header
                tmp_results_df.to_csv(output_file_name, mode='a', header=False,index=False)
        except:
            print('something went wrong and could not write the results to file!\n',
                 'You may abort and see what can be done.\n',
                 'Or wait to see the the final results in memory.')
    mean = results.mean(axis=0)
    maxx = results.max(axis=0)
    print(mean)
    print(maxx)
    # =============================================================================
    # K-means with init='k-means++'
    # =============================================================================
    print('\n- k-means++ -----------------------')
    for fold in tqdm(range(20)):
        gc.collect()
        seed = randint(0,10**5)
        model = KMeans(n_clusters=n_clusters,n_init=20,init='k-means++', random_state=seed).fit(X)
        predicted_labels = model.labels_
        try:
            tmp_results = ['k-means++','seed '+str(seed)]+evaluate(X,Y,predicted_labels)
            tmp_results_s = pd.Series(tmp_results, index = results.columns)
            tmp_results_df = results_template.copy()
            tmp_results_df = tmp_results_df.append(tmp_results_s, ignore_index=True)
            results = results.append(tmp_results_s, ignore_index=True)
        except:
            print('Some error happened, skipping ',fold)
        
        print('writing the fold results to file')
        # if file does not exist write header 
        try:
            if not os.path.isfile(output_file_name):
                tmp_results_df.to_csv(output_file_name, header=column_names,index=False)
            else: # else it exists so append without writing the header
                tmp_results_df.to_csv(output_file_name, mode='a', header=False,index=False)
        except:
            print('something went wrong and could not write the results to file!\n',
                 'You may abort and see what can be done.\n',
                 'Or wait to see the the final results in memory.')
    mean = results.mean(axis=0)
    maxx = results.max(axis=0)
    print(mean)
    print(maxx)
    # =============================================================================
    # Agglomerative
    # =============================================================================
    print('\n- Agglomerative -----------------------')
    for fold in tqdm(range(4)):
        gc.collect()
        model = AgglomerativeClustering(n_clusters=n_clusters,linkage='ward').fit(X)
        predicted_labels = model.labels_
        try:
            tmp_results = ['Agglomerative','ward']+evaluate(X,Y,predicted_labels)
            tmp_results_s = pd.Series(tmp_results, index = results.columns)
            tmp_results_df = results_template.copy()
            tmp_results_df = tmp_results_df.append(tmp_results_s, ignore_index=True)
            results = results.append(tmp_results_s, ignore_index=True)
        except:
            print('Some error happened, skipping ',fold)
        
        print('writing the fold results to file')
        # if file does not exist write header 
        try:
            if not os.path.isfile(output_file_name):
                tmp_results_df.to_csv(output_file_name, header=column_names,index=False)
            else: # else it exists so append without writing the header
                tmp_results_df.to_csv(output_file_name, mode='a', header=False,index=False)
        except:
            print('something went wrong and could not write the results to file!\n',
                 'You may abort and see what can be done.\n',
                 'Or wait to see the the final results in memory.')
    mean = results.mean(axis=0)
    maxx = results.max(axis=0)
    print(mean)
    print(maxx)
    # =============================================================================
    # DBSCAN
    # =============================================================================
    eps=0.000001
    print('\n- DBSCAN -----------------------')
    for fold in tqdm(range(19)):
        gc.collect()
        eps = eps+0.05
        model = DBSCAN(eps=eps, min_samples=10,n_jobs=15).fit(X)
        predicted_labels = model.labels_
        try:
            tmp_results = ['DBSCAN','eps '+str(eps)]+evaluate(X,Y,predicted_labels)
            tmp_results_s = pd.Series(tmp_results, index = results.columns)
            tmp_results_df = results_template.copy()
            tmp_results_df = tmp_results_df.append(tmp_results_s, ignore_index=True)
            results = results.append(tmp_results_s, ignore_index=True)
        except:
            print('Some error happened, skipping ',fold)
        
        print('writing the fold results to file')
        # if file does not exist write header 
        try:
            if not os.path.isfile(output_file_name):
                tmp_results_df.to_csv(output_file_name, header=column_names,index=False)
            else: # else it exists so append without writing the header
                tmp_results_df.to_csv(output_file_name, mode='a', header=False,index=False)
        except:
            print('something went wrong and could not write the results to file!\n',
                 'You may abort and see what can be done.\n',
                 'Or wait to see the the final results in memory.')
    mean = results.mean(axis=0)
    maxx = results.max(axis=0)
    print(mean)
    print(maxx)

    # =============================================================================
    # Save to disk
    # =============================================================================
    # print('Writing to disk...')
    results_df = pd.DataFrame(results)
    # results_df.to_csv(output_file_name,index=False)
    print('Done.')
    return results_df

## Evaluation of the results

In [ ]:
import pandas as pd
datapath = '/mnt/6016589416586D52/Users/z5204044/GoogleDrive/GoogleDrive/Data/' #C1314
# datapath = '/mnt/16A4A9BCA4A99EAD/GoogleDrive/Data/' #Zen
data_address =  datapath+"Corpus/KPRIS/embeddings/deflemm/performance/bert orig-large corpus.csv clustering results"
df = pd.read_csv(data_address)
max1 = df.groupby(['Method'], sort=False).max()
max2 = df.groupby(['Method']).agg({'NMI': 'max','AMI':'max','ARI':'max'})
max3 = df[df.groupby(['Method'])['NMI'].transform(max) == df['NMI']]
min3 = df[df.groupby(['Method'])['NMI'].transform(min) == df['NMI']]